In [8]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim, utils
from torchvision import models, datasets, transforms

import PIL.Image as Image

from tqdm import tqdm
import random
import cv2
import os
import time

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20,20)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using device ...", device)

Using device ... cuda


In [9]:
down_scale = 1

transforms_train = transforms.Compose([
    transforms.ColorJitter(brightness=0.1,contrast=0.2,saturation=0.2,hue=0.1),
    transforms.RandomAffine(360,scale=[1.15,1.25]),
    transforms.CenterCrop(448),
    transforms.Resize(224),
    transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    transforms.ColorJitter(brightness=0.1,contrast=0.2,saturation=0.2,hue=0.1),
    transforms.RandomAffine(360,scale=[1.15,1.25]),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
    
# dataset = MosquitoDatasets("/data/MosquitoDL/TrainVal/", transforms=mos_transforms)
train_dataset = datasets.ImageFolder("~/dataset/MosquitoDL2020/MosquitoDL2X/train", transform=transforms_train)
train_loader = utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

# dataset = MosquitoDatasets("/data/MosquitoDL/TrainVal/", transforms=mos_transforms)
# test_dataset = datasets.ImageFolder("/media/data/MosquitoDL/Test", transform=transforms_test)
test_dataset = datasets.ImageFolder("~/dataset/MosquitoDL2020/MosquitoDL2X/test", transform=transforms_test)
test_loader = utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=4)

# Show samples

batch_sample = next(iter(train_loader))
print(batch_sample[0].shape)

image_samples = torchvision.utils.make_grid(batch_sample[0],nrow=8).permute((1,2,0))

fig, ax = plt.subplots()
ax.axis('off')
ax.imshow(image_samples)
ax.set_title(f"Input Samples ↓x{down_scale}")
plt.savefig("./batch_samples.jpg")

batch_sample = next(iter(test_loader))
image_samples = torchvision.utils.make_grid(batch_sample[0],nrow=8).permute((1,2,0))

fig, ax = plt.subplots()
ax.axis('off')
ax.imshow(image_samples)
ax.set_title(f"Input Samples ↓x{down_scale}")
plt.savefig("./batch_samples.jpg")

NameError: name 'init_scale' is not defined

In [10]:
def get_model(model, pretrained=False, num_classes = 6):
    if model == 'resnet101':
        net = models.resnet101(pretrained=pretrained)
        net.fc = nn.Linear(in_features=2048, out_features=num_classes)
    elif model == 'resnet50':
        net = models.resnet50(pretrained=pretrained)
        net.fc = nn.Linear(in_features=2048, out_features=num_classes)
    elif model == 'resnet34':
        net = models.resnet34(pretrained=pretrained)
        net.fc = nn.Linear(in_features=512, out_features=num_classes)
    elif model == 'resnet18':
        net = models.resnet18(pretrained=pretrained)
        net.fc = nn.Linear(in_features=512, out_features=num_classes)
    return net
model_name = 'resnet18'
net_t = get_model(model_name, pretrained=True, num_classes=6)
net_t = net_t.to(device)
net_t = nn.DataParallel(net_t)

In [11]:
def train(net, train_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(),lr=1e-4, weight_decay=4e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, 50, gamma=0.1)
    
    train_avg_loss = 0
    n_count = 0
    n_corrects = 0

    net.train()

    for j, data in enumerate(train_loader):
        batch, label = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        pred = net(batch)

        loss = criterion(pred, label)
        train_avg_loss += loss

        n_corrects += torch.sum(torch.argmax(pred, dim=1) == label).item()
        n_count += label.shape[0]

        loss.backward()
        optimizer.step()

    train_accuracy = n_corrects/n_count
    train_avg_loss /= n_count

    return train_accuracy, train_avg_loss, net

In [12]:
def evaluate(net ,test_loader):
    net.eval()
    
    n_count = 0
    n_corrects = 0

    for j, data in enumerate(test_loader):


        batch, label = data[0].to(device), data[1].to(device)

        pred = net(batch)

        n_corrects += torch.sum(torch.argmax(pred, dim=1) == label).item()
        n_count += label.shape[0]

    test_accuracy = n_corrects/n_count
    
    return test_accuracy, net

# Better quality image sample test
- 448 -> 224 : Test 61%
- 224 -> 224 : Test ??%

In [6]:
def train_and_eval(net, epochs, train_loader, test_loader, save_name='default.pth'):
    print("─── Start Training & Evalutation ───")
    
    best_accuracy = 0
    best_model = None
    
    for i in range(epochs):
        print(f"┌ Epoch ({i}/{epochs-1})")
        
        time_s = time.time()

        train_acc, loss, net = train(net, train_loader)
        print(f"├── Training Loss : {loss:.4f}")
        print(f'├── Training accuracy : {train_acc*100:.2f}%')
        print("│ Testing ...")
        test_acc, net = evaluate(net, test_loader)
        print(f'└── Testing accuracy : {test_acc*100:.2f}%')

        

        if test_acc > best_accuracy:
            print(f"  └──> Saving the best model to \"{save_name}\"")
            best_accuracy = test_acc
            best_model = net.module.state_dict()
            model_dict = {'acc':best_accuracy, 'net':best_model}
            torch.save(model_dict, save_name)

        time_e = time.time()
        
        epoch_t = time.gmtime(time_e - time_s)
        est_t = time.gmtime((time_e - time_s) * epochs-1-i)
        print(f"Epoch time : {epoch_t.tm_hour}[h] {epoch_t.tm_min}[m] {epoch_t.tm_sec}[s]")
        print(f"Estimated time : {est_t.tm_hour}[h] {est_t.tm_min}[m] {est_t.tm_sec}[s]")
        print("\n")
            
    return best_accuracy, best_model
        
epochs = 150

accuracy, net_t = train_and_eval(net_t, epochs, train_loader, test_loader, save_name=f"./{model_name}_x{down_scale}.pth")


149)
├── Training Loss : 0.0002
├── Training accuracy : 99.52%
│ Testing ...
└── Testing accuracy : 76.89%
Epoch time : 0[h] 0[m] 35[s]
Estimated time : 1[h] 27[m] 40[s]


┌ Epoch (43/149)
├── Training Loss : 0.0002
├── Training accuracy : 99.64%
│ Testing ...
└── Testing accuracy : 85.00%
  └──> Saving the best model to "./resnet18_x1.pth"
Epoch time : 0[h] 0[m] 35[s]
Estimated time : 1[h] 27[m] 32[s]


┌ Epoch (44/149)
├── Training Loss : 0.0003
├── Training accuracy : 99.37%
│ Testing ...
└── Testing accuracy : 76.11%
Epoch time : 0[h] 0[m] 35[s]
Estimated time : 1[h] 27[m] 7[s]


┌ Epoch (45/149)
├── Training Loss : 0.0002
├── Training accuracy : 99.33%
│ Testing ...
└── Testing accuracy : 76.22%
Epoch time : 0[h] 0[m] 35[s]
Estimated time : 1[h] 27[m] 30[s]


┌ Epoch (46/149)
├── Training Loss : 0.0003
├── Training accuracy : 99.48%
│ Testing ...
└── Testing accuracy : 72.40%
Epoch time : 0[h] 0[m] 35[s]
Estimated time : 1[h] 27[m] 10[s]


┌ Epoch (47/149)
├── Training Loss : 0.00